In [1]:
# Web Scraping 

In [2]:
from bs4 import BeautifulSoup
import requests
import csv

In [3]:
url="https://www.imdb.com/chart/top/?ref_=nv_mv_250"

In [4]:
page= requests.get(url)

In [5]:
page

<Response [200]>

In [6]:
soup= BeautifulSoup(page.text, "html.parser")

In [7]:
soup


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
<style>
                body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
            </style>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>IMDb Top 250 - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>


In [8]:
raw_html = soup.find("tbody", {"class": "lister-list"}).findAll("tr")

In [9]:
def main_fun():
    movie_list = []
    for html in raw_html:
        title = html.find("td", {"class": "titleColumn"}).find("a").get_text()
        year = html.find("td", {"class": "titleColumn"}).find("span").getText()[1:5]
        rating= html.find("td", {"class": "ratingColumn imdbRating"}).find("strong").get_text()
        rating= float(rating)
        
        raw_list= [title, year, rating]
        movie_list.append(raw_list)
        
        
        create_csv(movie_list)
        

        
        

In [10]:
def create_csv(movies):
    with open('movies.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        row=['Name','Year','Rating']
        writer.writerow(row)
        for x in range(len(movies)):
            row = movies[x]
            writer.writerow(row)
        csvfile.close()
            

In [12]:
main_fun()

UnicodeEncodeError: 'cp949' codec can't encode character '\xe8' in position 8: illegal multibyte sequence

In [ ]:
## NYP times API

In [13]:
from config import api_key
import requests
import json
import pandas as pd
from pprint import pprint
import time

In [14]:
movie_names = []
from csv import reader
with open("movies_utf_8.csv", 'r') as reading_object:
    csv_reader = reader(reading_object)
    next(csv_reader)
    for i in csv_reader:
        movie_names.append(i[0])

In [15]:
movie_names

['The Shawshank Redemption',
 'The Godfather',
 'The Godfather: Part II',
 'The Dark Knight',
 '12 Angry Men',
 "Schindler's List",
 'The Lord of the Rings: The Return of the King',
 'Pulp Fiction',
 'Il buono, il brutto, il cattivo',
 'The Lord of the Rings: The Fellowship of the Ring',
 'Fight Club',
 'Forrest Gump',
 'Inception',
 'Star Wars: Episode V - The Empire Strikes Back',
 'The Lord of the Rings: The Two Towers',
 'The Matrix',
 'Goodfellas',
 "One Flew Over the Cuckoo's Nest",
 'Shichinin no samurai',
 'Se7en',
 'La vita ? bella',
 'Cidade de Deus',
 'The Silence of the Lambs',
 "It's a Wonderful Life",
 'Star Wars',
 'Saving Private Ryan',
 'Hamilton',
 'Sen to Chihiro no kamikakushi',
 'The Green Mile',
 'Gisaengchung',
 'Interstellar',
 'L?on',
 'The Usual Suspects',
 'Seppuku',
 'The Lion King',
 'Back to the Future',
 'The Pianist',
 'Terminator 2: Judgment Day',
 'American History X',
 'Modern Times',
 'Psycho',
 'Gladiator',
 'City Lights',
 'The Departed',
 'The Int

In [17]:
url = 'https://api.nytimes.com/svc/movies/v2/reviews/search.json?'
response_list = []
alternate_list = []
movies_nyt = []
for x in movie_names:
    base_url = url + "query=" + str(x) + "&api-key=" + api_key
    response = requests.get(base_url).json()
    response_list.append(response)
    try:
        if response['results'] != []:
            alternate_list.append(response['results'][int(0)])
            movies_nyt.append(x)
    except:
        next
    time.sleep(6)

KeyError: 'results'

In [ ]:
no_result_count = 0
for x in response_list:
    if x['results'] == []:
        no_result_count += 1
print(no_result_count)

In [ ]:
with open('alternate.json', 'w', encoding='UTF-8') as hey:
    json.dump(alternate_list, hey, ensure_ascii=False, indent=4)

In [ ]:
thedf = pd.read_json('alternate.json')
thedf # 185 rows. missing 65 results from api.

In [ ]:
## Rotten Tomatoes

In [ ]:
from rotten_tomatoes_client import RottenTomatoesClient

meterclass = []
meterscore = []
movies_weget = []
movies_wedontget = []
for x in movie_names:
    try:
        result = RottenTomatoesClient.search(term=x, limit=1)
        if result['movies'][int(0)]['meterClass'] and result['movies'][int(0)]['meterScore']:
            meterclass.append(result['movies'][int(0)]['meterClass'])
            meterscore.append(result['movies'][int(0)]['meterScore'])
            movies_weget.append(x)
            print(x)
    except:
        movies_wedontget.append(x)
        next

In [ ]:
rot_df = pd.DataFrame({
    "movie_name":movies_weget,
    "meter_class" : meterclass,
    "meter_score" : meterscore
})

In [ ]:
rot_df.set_index('movie_name', inplace=True)

In [ ]:
rot_df